<a href="https://colab.research.google.com/github/zajack/zajack-CourseraProjects/blob/main/ALS%20recommend%20Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **WELCOME TO THIS NOTEBOOK**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Let's install pyspark

In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 76kB/s 
     |████████████████████████████████| 204kB 37.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=6fbf7ff8cee343502c14dc422b1b099f336d3aeaad72a186ce0e29bbb5a87856
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


Importing the modules

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max
from pyspark.ml.feature import  StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

Creating the spark session


In [ ]:
spark = SparkSession.builder.appName("lastfm").getOrCreate()

# Loading the dataset

In [ ]:
file_path = '/content/drive/MyDrive/dataset/listenings.csv'
df_listenings = spark.read.format('csv').option('header', True).option('InferSchema', True).load(file_path)
df_listenings.show()
# InferSchema - col names as in csv

+-----------+-------------+--------------------+---------------+--------------------+
|    user_id|         date|               track|         artist|               album|
+-----------+-------------+--------------------+---------------+--------------------+
|000Silenced|1299680100000|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|          Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|      Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      


# Cleaning tables 

In [ ]:
df_listenings= df_listenings.drop('date')
df_listenings.show()

+-----------+--------------------+---------------+--------------------+
|    user_id|               track|         artist|               album|
+-----------+--------------------+---------------+--------------------+
|000Silenced|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|            Acapella|          Kelis|            Acapella|
|000Silenced|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|               ObZen|      Meshuggah|               ObZen|
|000Silenced|   Yama's Messengers|         Gojira|The Way of All Flesh|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For No...|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For

In [ ]:
df_listenings = df_listenings.na.drop()
df_listenings.show()

+-----------+--------------------+---------------+--------------------+
|    user_id|               track|         artist|               album|
+-----------+--------------------+---------------+--------------------+
|000Silenced|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|            Acapella|          Kelis|            Acapella|
|000Silenced|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|               ObZen|      Meshuggah|               ObZen|
|000Silenced|   Yama's Messengers|         Gojira|The Way of All Flesh|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For No...|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For

In [ ]:
# in pandas we have shape, in pyspark: count - rows and
row_numbers = df_listenings.count()
column_numbers = len(df_listenings.columns)
print(row_numbers, column_numbers)

13758905 4



# Let's Perform some aggregation
to see how many times each user has listened to specific track


In [ ]:
df_listenings_agg = df_listenings.select('user_id', 'track').groupby('user_id', 'track').agg(count('*').alias('count')).orderBy('user_id')
df_listenings_agg.show()

+-------+--------------------+-----+
|user_id|               track|count|
+-------+--------------------+-----+
| --Seph|Chelsea Hotel - L...|    1|
| --Seph|Vestido Estampado...|    1|
| --Seph|          Paris 2004|    7|
| --Seph|Hungarian Rhapsod...|    1|
| --Seph| Air on the G String|    1|
| --Seph|        Window Blues|    1|
| --Seph|         The Embrace|    1|
| --Seph|   Summa for Strings|    1|
| --Seph|      Hour for magic|    2|
| --Seph|     The Way We Were|    1|
| --Seph| White Winter Hymnal|    3|
| --Seph|Hungarian Dance No 5|    1|
| --Seph|       Phantom Pt II|    1|
| --Seph|               Leloo|    1|
| --Seph|Belina (Original ...|    1|
| --Seph|  California Waiting|    1|
| --Seph|Virus (Luke Fair ...|    1|
| --Seph|              Monday|    1|
| --Seph|       Life On Mars?|    1|
| --Seph|Airplanes [feat H...|    1|
+-------+--------------------+-----+
only showing top 20 rows



In [ ]:
r_aggr = df_listenings_agg.count()
col_aggr = len(df_listenings_agg.columns)
print(r_aggr,col_aggr)

9930128 3


In [ ]:
df_listenings_agg = df_listenings_agg.limit(20000)

# Let's convert the user id and track columns into unique integers




In [ ]:
#String indexer - convert text into a number
indexer = [StringIndexer(inputCol=col , outputCol=col+'_index').fit(df_listenings_agg) for col in list(set(df_listenings_agg.columns) - set(['count']))
]

pipeline = Pipeline(stages=indexer)

data = pipeline.fit(df_listenings_agg).transform(df_listenings_agg)
data.show()


+-------+--------------------+-----+-----------+-------------+
|user_id|               track|count|track_index|user_id_index|
+-------+--------------------+-----+-----------+-------------+
| --Seph| White Winter Hymnal|    3|       59.0|         69.0|
| --Seph|Virus (Luke Fair ...|    1|    15896.0|         69.0|
| --Seph|Airplanes [feat H...|    1|      519.0|         69.0|
| --Seph|Belina (Original ...|    1|     3278.0|         69.0|
| --Seph|              Monday|    1|      334.0|         69.0|
| --Seph|Hungarian Dance No 5|    1|     7558.0|         69.0|
| --Seph|       Life On Mars?|    1|     1161.0|         69.0|
| --Seph|  California Waiting|    1|      197.0|         69.0|
| --Seph|       Phantom Pt II|    1|     1377.0|         69.0|
| --Seph|   Summa for Strings|    1|    13739.0|         69.0|
| --Seph|      Hour for magic|    2|     7495.0|         69.0|
| --Seph|Hungarian Rhapsod...|    1|     7559.0|         69.0|
| --Seph|     The Way We Were|    1|    14960.0|       

In [ ]:
data = data.select('user_id_index', 'track_index', 'count').orderBy('user_id_index')

In [ ]:
data.show()

+-------------+-----------+-----+
|user_id_index|track_index|count|
+-------------+-----------+-----+
|          0.0|    10943.0|    1|
|          0.0|    11628.0|    2|
|          0.0|     1349.0|    1|
|          0.0|      381.0|    1|
|          0.0|     8692.0|    1|
|          0.0|     6899.0|    1|
|          0.0|    14044.0|    1|
|          0.0|    15513.0|    1|
|          0.0|    11978.0|    2|
|          0.0|    15176.0|    1|
|          0.0|     8305.0|    1|
|          0.0|    13722.0|    1|
|          0.0|    10620.0|    1|
|          0.0|     4424.0|    1|
|          0.0|    16732.0|    1|
|          0.0|    10630.0|    1|
|          0.0|    12169.0|    1|
|          0.0|     4117.0|    1|
|          0.0|    10336.0|    1|
|          0.0|    16829.0|    1|
+-------------+-----------+-----+
only showing top 20 rows



# Train and Test data

In [ ]:
(training, test) = data.randomSplit([0.5, 0.5])

# Let's Create our Model

In [ ]:
USERID = 'user_id_index'
TRACK = 'track_index'
COUNT = 'count'

als = ALS(maxIter=5, regParam=0.01, userCol=USERID, itemCol = TRACK, ratingCol = COUNT)
model = als.fit(training)

predictions = model.transform(test)



# Generate top 10 Track recommendations for each user

In [ ]:
recs = model.recommendForAllUsers(10)

In [ ]:
recs.show()

+-------------+--------------------+
|user_id_index|     recommendations|
+-------------+--------------------+
|          148|[[7849, 8.290683]...|
|           31|[[235, 11.199126]...|
|           85|[[4461, 16.786757...|
|          137|[[4461, 10.044955...|
|           65|[[8821, 10.209071...|
|           53|[[182, 12.976642]...|
|          133|[[14826, 17.99709...|
|           78|[[8821, 14.934841...|
|          108|[[8175, 7.9877386...|
|           34|[[4461, 15.07089]...|
|          101|[[2379, 11.974902...|
|          115|[[12399, 9.181116...|
|          126|[[568, 13.926398]...|
|           81|[[235, 9.298625],...|
|           28|[[8821, 8.762349]...|
|           76|[[7849, 7.6234875...|
|           26|[[4461, 24.832445...|
|           27|[[3523, 7.5699906...|
|           44|[[4461, 13.394168...|
|          103|[[4461, 22.810787...|
+-------------+--------------------+
only showing top 20 rows



In [ ]:
recs.take(1)

[Row(user_id_index=148, recommendations=[Row(track_index=7849, rating=8.290682792663574), Row(track_index=14826, rating=8.290682792663574), Row(track_index=11299, rating=8.263848304748535), Row(track_index=6154, rating=7.575194358825684), Row(track_index=14108, rating=7.575194358825684), Row(track_index=9557, rating=7.369495391845703), Row(track_index=8496, rating=7.369495391845703), Row(track_index=4619, rating=7.369495391845703), Row(track_index=2379, rating=7.125980377197266), Row(track_index=182, rating=6.3732218742370605)])]